In [1]:
import logging
from tqdm import tqdm

import torch

import config
import modules.metrics_mAP_aladdin as aladdin_mAP
import modules.dataloaders as data_loaders
import modules.models_brevitas_noComp_fixed_point as models_bnn_fxpoint_no_comp
import modules.models_brevitas_fixed_point as models_bnn_fxpoint
import modules.utils as utils
import modules.metrics as metrics

In [2]:
from brevitas.export import export_onnx_qcdq

# Logger and Folders to Save Plots

In [3]:
log_path = './mAP_&_pr_curves/brevitas_no_comp_w4W2h8b4/'

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('Simple BED Detector.\n' +
           '\tNo Sigmoid. No Softmax. Permute out of the model.\n' +
            '\tDFire and FASDD UAV and CV.\n' +
            f'\tMax Obj = {config.MAX_OBJ}.\n' +
            f'\tIOU Threshold = {config.IOU_THRESHOLD}.\n' +
            f'\tNMS IOU Threshold = {config.NMS_IOU_THRESHOLD}.\n' +
            '\tNMS IOU Threshold does not change too much, so it is not tested.')

logger.info('\nBrevitas Config:')
logger.info(f'\tFixed Point: {config.FIXED_POINT}')
logger.info(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
logger.info(f'\tHead Weights Bit Width: {config.HEAD_WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
logger.info(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')

# Datasets DFire

## DFire

In [4]:
dfire_loader = data_loaders.get_dfire_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test dataset len: 3983


## FASDD UAV

In [5]:
fasdd_uav_loader = data_loaders.get_fasdd_uav_val_loader()


TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648


## FASDD CV

In [6]:
fasdd_cv_loader = data_loaders.get_fasdd_cv_val_loader()


TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523


## DFire, FASDD UAV and CV

In [7]:
full_loader = data_loaders.get_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test DFire dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 7631
Concatenate with FASDD CV dataset
Test dataset len: 23154


# Load Model

In [8]:
model = models_bnn_fxpoint_no_comp.FIXED_POINT_QUANT_SIMPLE_BED_DETECTOR(
    weight_bw = config.WEIGHTS_BIT_WIDTH, 
    big_layers_weight_bw = config.BIG_LAYERS_WEIGHTS_BIT_WIDTH,
    head_weight_bw = config.HEAD_WEIGHTS_BIT_WIDTH,
    act_bw = config.ACTIVATIONS_BIT_WIDTH, 
    bias_bw = config.BIAS_BIT_WIDTH,         
    ).to(config.DEVICE)

# model = models_bnn_fxpoint.FIXED_POINT_QUANT_PRUNED_AFTER_SVD_BED_DETECTOR(
#         weight_bw = config.WEIGHTS_BIT_WIDTH, 
#         big_layers_weight_bw = config.BIG_LAYERS_WEIGHTS_BIT_WIDTH,
#         head_weight_bw = config.HEAD_WEIGHTS_BIT_WIDTH,
#         act_bw = config.ACTIVATIONS_BIT_WIDTH, 
#         bias_bw = config.BIAS_BIT_WIDTH,         
#     ).to(config.DEVICE)

### Load Weights

In [9]:
# _______________________________ NO COMP _______________________________ #
weights_path = './experiments/test_50_no_comp_brevitas_fixed_point_w4W2H8a8b4/weights/'
weights_file = weights_path + 'BED_detector__best_mAP=0.6170__epoch=129.pt'

# _______________________________ AIMET w8a8b8 _______________________________ #
# weights_path = './experiments/test_42_aimet_brevitas_fixed_point_w8a8b8/weights/'
# weights_file = weights_path + 'BED_detector__best_mAP=0.6293__epoch=94.pt'

# _______________________________ AIMET w5W4h8b5 _______________________________ #
# weights_path = './experiments/test_451_aimet_brevitas_fixed_point_w5W4H8a8b5_more_train/weights/'
# weights_file = weights_path + 'BED_detector__best_mAP=0.6167__epoch=49.pt'


logger.info(f'\nWeights file: {weights_file}')

utils.load_checkpoint(
    model_path=weights_file, 
    model=model, 
    optimizer=None, 
    scheduler=None, 
    device=config.DEVICE)

Loading Model. Trained during 129 epochs


129

### Evaluation Mode

In [10]:
model.eval()

FIXED_POINT_QUANT_SIMPLE_BED_DETECTOR(
  (model): Sequential(
    (input0): QuantIdentity(
      (input_quant): ActQuantProxyFromInjector(
        (_zero_hw_sentinel): StatelessBuffer()
      )
      (act_quant): ActQuantProxyFromInjector(
        (_zero_hw_sentinel): StatelessBuffer()
        (fused_activation_quant_proxy): FusedActivationQuantProxy(
          (activation_impl): Identity()
          (tensor_quant): RescalingIntQuant(
            (int_quant): IntQuant(
              (float_to_int_impl): RoundSte()
              (tensor_clamp_impl): TensorClamp()
              (delay_wrapper): DelayWrapper(
                (delay_impl): _NoDelay()
              )
            )
            (scaling_impl): ConstScaling(
              (restrict_clamp_scaling): _RestrictClampValue(
                (clamp_min_ste): ScalarClampMinSte()
                (restrict_value_impl): PowerOfTwoRestrictValue(
                  (float_to_int_impl): CeilSte()
                  (power_of_two): PowerOfTwo()

# Export to ONNX

In [11]:
onnx_folder = './onnx_models/'

In [12]:
export_onnx_qcdq(
    model, 
    torch.randn(1, 3, config.IMG_H, config.IMG_W).to(config.DEVICE), 
    export_path=onnx_folder+'w4W2h8b4__bed_detector___no_comp__fixed_point__qcdq.onnx')

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/brevitas/export/onnx/standard/manager.py:26: UserWarning: ONNX opset version set to 13, override with opset_version=
  warnings.warn(f"ONNX opset version set to {DEFAULT_OPSET}, override with {ka}=")


### CPU

In [13]:
model.to('cpu')
export_onnx_qcdq(
    model, 
    torch.randn(1, 3, config.IMG_H, config.IMG_W), 
    export_path=onnx_folder+'w4W2h8b4__bed_detector___no_comp__fixed_point__qcdq__CPU.onnx')

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/brevitas/export/onnx/standard/manager.py:26: UserWarning: ONNX opset version set to 13, override with opset_version=
  warnings.warn(f"ONNX opset version set to {DEFAULT_OPSET}, override with {ka}=")


### Model Back to Device

In [14]:
model.to(config.DEVICE)

FIXED_POINT_QUANT_SIMPLE_BED_DETECTOR(
  (model): Sequential(
    (input0): QuantIdentity(
      (input_quant): ActQuantProxyFromInjector(
        (_zero_hw_sentinel): StatelessBuffer()
        (export_handler): None
      )
      (act_quant): ActQuantProxyFromInjector(
        (_zero_hw_sentinel): StatelessBuffer()
        (fused_activation_quant_proxy): FusedActivationQuantProxy(
          (activation_impl): Identity()
          (tensor_quant): RescalingIntQuant(
            (int_quant): IntQuant(
              (float_to_int_impl): RoundSte()
              (tensor_clamp_impl): TensorClamp()
              (delay_wrapper): DelayWrapper(
                (delay_impl): _NoDelay()
              )
            )
            (scaling_impl): ConstScaling(
              (restrict_clamp_scaling): _RestrictClampValue(
                (clamp_min_ste): ScalarClampMinSte()
                (restrict_value_impl): PowerOfTwoRestrictValue(
                  (float_to_int_impl): CeilSte()
               

# Print values

In [15]:
def print_metrics(mAP_metrics):
    mAP, avg_prec, cls_prec, cls_rec = mAP_metrics
    
    mAP_str = "mAP @0.50"
    smoke = "Smoke"
    fire = "Fire"
    
    print(f'{mAP_str:<12}' + f'{mAP:.4f}')
    print('Average Precision')
    print(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    print('Class Precision')
    print(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    print('Class Recall')
    print(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')
    
    logger.info(f'{mAP_str:<12}' + f'{mAP:.4f}')
    logger.info('Average Precision')
    logger.info(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    logger.info(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    logger.info('Class Precision')
    logger.info(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    logger.info(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    logger.info('Class Recall')
    logger.info(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    logger.info(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')

# Calculate mAP DFire

In [16]:
pred_boxes_1, true_boxes_1 = aladdin_mAP.get_bboxes(
    dfire_loader,
    model)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [00:06<00:00,  9.20it/s]


In [17]:
dfire_metrics = aladdin_mAP.mAP(
    log_path=log_path + 'dfire/',
    pred_boxes=pred_boxes_1,
    true_boxes=true_boxes_1)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.14it/s]


In [18]:
logger.info('\n\nDFire Aladdin Metrics:\n')
print_metrics(dfire_metrics)

mAP @0.50   0.3660
Average Precision
- Smoke     0.4061
- Fire      0.3258
Class Precision
- Smoke     0.5371
- Fire      0.5092
Class Recall
- Smoke     0.5238
- Fire      0.4069


## Compare to Torchmetrics

In [19]:
dfire_torchmetrics = metrics.torchmetrics_mAP(
    dfire_loader,
    model)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [00:05<00:00, 12.00it/s]


In [20]:
print(dfire_torchmetrics)
logger.info(f'\nDFire Torchmetrics: \n{dfire_torchmetrics}')

{'mAP': tensor(0.3692), 'AP': [0.4105823040008545, 0.3279108703136444], 'AR': [0.5237603187561035, 0.4068838357925415]}


# Calculate mAP FASDD UAV

In [21]:
pred_boxes_2, true_boxes_2 = aladdin_mAP.get_bboxes(
    fasdd_uav_loader,
    model)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:15<00:00,  3.77it/s]


In [22]:
fasdd_uav_metrics = aladdin_mAP.mAP(
    log_path=log_path + 'fasdd_uav/',
    pred_boxes=pred_boxes_2,
    true_boxes=true_boxes_2)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.82it/s]


In [23]:
logger.info('\n\nFASDD UAV Aladdin Metrics:\n')
print_metrics(fasdd_uav_metrics)

mAP @0.50   0.5667
Average Precision
- Smoke     0.6313
- Fire      0.5021
Class Precision
- Smoke     0.7889
- Fire      0.8458
Class Recall
- Smoke     0.6539
- Fire      0.5157


### Compare to Torchmetrics

In [24]:
fasdd_uav_torchmetrics = metrics.torchmetrics_mAP(
    fasdd_uav_loader,
    model)

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:15<00:00,  3.72it/s]


In [25]:
print(fasdd_uav_torchmetrics)
logger.info(f'\nFASDD UAV Torchmetrics: \n{fasdd_uav_torchmetrics}')

{'mAP': tensor(0.5663), 'AP': [0.6309635043144226, 0.5017337203025818], 'AR': [0.6538830399513245, 0.5156679749488831]}


# Calculate mAP FASDD CV

In [26]:
pred_boxes_3, true_boxes_3 = aladdin_mAP.get_bboxes(
    fasdd_cv_loader,
    model)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [00:23<00:00, 10.13it/s]


In [27]:
fasdd_cv_metrics = aladdin_mAP.mAP(
    log_path=log_path + 'fasdd_cv/',
    pred_boxes=pred_boxes_3,
    true_boxes=true_boxes_3)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.83s/it]


In [28]:
logger.info('\n\nFASDD CV Aladdin Metrics:\n')
print_metrics(fasdd_cv_metrics)

mAP @0.50   0.6782
Average Precision
- Smoke     0.6595
- Fire      0.6969
Class Precision
- Smoke     0.7177
- Fire      0.7143
Class Recall
- Smoke     0.7271
- Fire      0.7428


### Compare to Torchmetrics

In [29]:
fasdd_cv_torchmetrics = metrics.torchmetrics_mAP(
    fasdd_cv_loader,
    model)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [00:18<00:00, 13.16it/s]


In [30]:
print(fasdd_cv_torchmetrics)
logger.info(f'\nFASDD CV Torchmetrics: \n{fasdd_cv_torchmetrics}')

{'mAP': tensor(0.6772), 'AP': [0.6573913097381592, 0.6970455646514893], 'AR': [0.7271081209182739, 0.7427741885185242]}


# Calculate mAP FULL DS

In [31]:
pred_boxes, true_boxes = aladdin_mAP.get_bboxes(
    full_loader,
    model)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [00:43<00:00,  8.24it/s]


In [32]:
full_ds_metrics = aladdin_mAP.mAP(
    log_path=log_path + 'full_ds/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.39s/it]


In [33]:
logger.info('\n\nFull Dataset Aladdin Metrics:\n')
print_metrics(full_ds_metrics)

mAP @0.50   0.6151
Average Precision
- Smoke     0.6199
- Fire      0.6103
Class Precision
- Smoke     0.6987
- Fire      0.7020
Class Recall
- Smoke     0.6804
- Fire      0.6528


### Compare to Torchmetrics

In [34]:
full_ds_torchmetrics = metrics.torchmetrics_mAP(
    full_loader,
    model)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [00:36<00:00,  9.78it/s]


In [35]:
print(full_ds_torchmetrics)
logger.info(f'\nFull Dataset Torchmetrics: \n{full_ds_torchmetrics}')

{'mAP': tensor(0.6170), 'AP': [0.6227277517318726, 0.6111970543861389], 'AR': [0.6803808212280273, 0.6527764797210693]}
